In [ ]:
import ray
from ray.job_submission import JobSubmissionClient
import time
from ray.runtime_env import RuntimeEnv

# Ray cluster information
ray_head_ip = "kuberay-head-svc.kuberay.svc.cluster.local"
ray_head_port = 8265
ray_address = f"http://{ray_head_ip}:{ray_head_port}"

# Submit Ray job using JobSubmissionClient
client = JobSubmissionClient(ray_address)
job_id = client.submit_job(
    entrypoint="python mandlebrot.py",
    runtime_env={
        "working_dir": "./",
        "pip": ["piexif","ipyplot","whylogs[image,whylabs]"],
        "env_vars": {"http_proxy":"http://10.78.90.46:80","https_proxy":"http://10.78.90.46:80"}
    },
    entrypoint_num_cpus=3
)

print(client.__dict__)
print(f"Ray job submitted with job_id: {job_id}")

# Wait for a while to let the jobs run
time.sleep(1)

job_status = client.get_job_status(job_id)
get_job_logs = client.get_job_logs(job_id)
get_job_info = client.get_job_info(job_id)
print(f"Ray job status for job_id {job_id}: {job_status}")
print(f"Ray job logs for job_id {job_id}: {get_job_logs}")
print(f"Ray job info for job_id {job_id}: {get_job_info}")
async for lines in client.tail_job_logs(job_id):
    print(lines, end="") 

# Disconnect from the Ray cluster
ray.shutdown()